In [23]:
x = [
    1 2 3
    3 2 6
    7 8 9
]

size(x)

z = zeros(Float64,size(x))

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [10]:
abstract type Node end

struct InputNode <: Node
    value::Matrix{Float64}
    grad::Matrix{Float64}
end

struct OperationNode <: Node
    op::Function
    inputs::Vector{Node}
    value::Matrix{Float64}
    grad::Matrix{Float64}
end

function InputNode(value::Matrix{Float64})
    InputNode(value, zeros(size(value)))
end

function OperationNode(op::Function, inputs::Vector{Node})
    OperationNode(op, inputs, zeros(size(inputs[1].value)), zeros(size(inputs[1].value)))
end


OperationNode

In [26]:
x_d = [
    1 2 3
    3 2 6
    7 8 9.0
]
x = InputNode(x_d)

InputNode([1.0 2.0 3.0; 3.0 2.0 6.0; 7.0 8.0 9.0], [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0])

In [11]:
function add(x::Node, y::Node)
    return OperationNode((a, b) -> a .+ b, [x, y])
end

function mul(x::Node, y::Node)
    return OperationNode((a, b) -> a * b, [x, y])
end

function eval(node::InputNode)
    return node.value
end

function eval(node::OperationNode)
    input_values = map(eval, node.inputs)
    return node.op(input_values...)
end


eval (generic function with 3 methods)

In [12]:
struct RNNCell
    Wxh::Matrix{Float64}  # Wagi dla wejścia
    Whh::Matrix{Float64}  # Wagi dla stanu ukrytego
    Why::Matrix{Float64}  # Wagi dla wyjścia
    bh::Vector{Float64}   # Bias dla stanu ukrytego
    by::Vector{Float64}   # Bias dla wyjścia
end

function RNNCell(input_dim::Int, hidden_dim::Int, output_dim::Int)
    RNNCell(randn(hidden_dim, input_dim), randn(hidden_dim, hidden_dim),
            randn(output_dim, hidden_dim), randn(hidden_dim), randn(output_dim))
end


RNNCell

In [35]:
function rnn_forward(cell::RNNCell, inputs::Vector{InputNode})
    h = zeros(size(cell.Whh))  # Początkowy stan ukryty (kolumna)
    outputs = Vector{Matrix{Float64}}(undef, length(inputs))
    states = Vector{Matrix{Float64}}(undef, length(inputs))

    for t in 1:length(inputs)
        h = tanh.(cell.Wxh * inputs[t].value .+ cell.Whh * h .+ cell.bh)
        y = cell.Why * h .+ cell.by
        states[t] = h
        outputs[t] = y
    end

    return outputs, states
end


rnn_forward (generic function with 1 method)

In [14]:
function rnn_backward(cell::RNNCell, inputs::Vector{InputNode}, states::Vector{Matrix{Float64}}, outputs::Vector{Matrix{Float64}}, targets::Vector{Matrix{Float64}}, learning_rate::Float64)
    dWxh, dWhh, dWhy = zeros(cell.Wxh), zeros(cell.Whh), zeros(cell.Why)
    dbh, dby = zeros(cell.bh), zeros(cell.by)
    dh_next = zeros(size(states[1]))

    for t in length(inputs):-1:1
        dy = outputs[t] - targets[t]
        dWhy += dy * states[t]'
        dby += dy
        dh = (cell.Why' * dy) + dh_next
        dh_raw = (1 .- states[t].^2) .* dh  # Gradient through tanh
        dbh += dh_raw
        dWxh += dh_raw * inputs[t].value'
        dWhh += dh_raw * (t > 1 ? states[t-1] : zeros(size(states[1])))'
        dh_next = cell.Whh' * dh_raw
    end

    # Update weights
    cell.Wxh -= learning_rate * dWxh
    cell.Whh -= learning_rate * dWhh
    cell.Why -= learning_rate * dWhy
    cell.bh -= learning_rate * dbh
    cell.by -= learning_rate * dby

    return cell
end


rnn_backward (generic function with 1 method)

In [40]:
# Parametry
input_dim = 3
hidden_dim = 5
output_dim = 2
seq_length = 10
learning_rate = 0.01

cell = RNNCell(input_dim, hidden_dim, output_dim)


inputs = [InputNode(rand(input_dim, 1)) for _ in 1:seq_length]
targets = [rand(output_dim, 1) for _ in 1:seq_length]


for epoch in 1:100
    println("Epoch: $epoch")
    outputs, states = rnn_forward(cell, inputs)
    println("straken")
    cell = rnn_backward(cell, inputs, states, outputs, targets, learning_rate)
end

Epoch: 1
straken


MethodError: MethodError: no method matching zeros(::Matrix{Float64})

Closest candidates are:
  zeros(!Matched::Union{Integer, AbstractUnitRange}...)
   @ Base array.jl:631
  zeros(!Matched::Type{T}, !Matched::Tuple{}) where T
   @ Base array.jl:640
  zeros(!Matched::Type{T}, !Matched::Tuple{Vararg{Integer, N}}) where {T, N}
   @ Base array.jl:635
  ...


In [31]:
# Parametry
input_dim = 3
hidden_dim = 5
output_dim = 2
seq_length = 10
learning_rate = 0.01

# Tworzymy RNN
cell = RNNCell(input_dim, hidden_dim, output_dim)

# Generujemy przykładowe dane wejściowe i wyjściowe
inputs = [InputNode(rand(input_dim, 1)) for _ in 1:seq_length]
targets = [rand(output_dim, 1) for _ in 1:seq_length]

# Trenujemy RNN przez kilka epok
for epoch in 1:100
    outputs, states = rnn_forward(cell, inputs)
    cell = rnn_backward(cell, inputs, states, outputs, targets, learning_rate)
end

# Wyświetlamy wyniki
println("Trained RNN output: ", outputs[end])


MethodError: MethodError: no method matching zeros(::Matrix{Float64})

Closest candidates are:
  zeros(!Matched::Union{Integer, AbstractUnitRange}...)
   @ Base array.jl:631
  zeros(!Matched::Type{T}, !Matched::Tuple{}) where T
   @ Base array.jl:640
  zeros(!Matched::Type{T}, !Matched::Tuple{Vararg{Integer, N}}) where {T, N}
   @ Base array.jl:635
  ...
